In [2]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

In [ ]:
!gdown 1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9

In [3]:
%cd deep-text-recognition-benchmark

/content/drive/MyDrive/deep-text-recognition-benchmark


In [ ]:
!unzip -qq plate_img-test.zip

In [4]:
!pip install lmdb
!pip3 install fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116933 sha256=8ec0f3cb69606696e554274d19d84731146cf79c22dd95c61fcd67d6a97f83d0
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [ ]:
import xml.etree.ElementTree as ET
import os

def is_latin_char(character):
    return character.isalpha() and ord(character) < 128

def is_latin_numeral(character):
    return character.isdigit() and ord(character) < 128

dataclass = ['train','validation']
dicts = {
        'B': 'ب',
        'P': 'پ',
        'T': 'ت',
        'Y': 'ث',
        'Z': 'ز',
        'X': 'ش',
        'E': 'ع',
        'F': 'ف',
        'K': 'ک',
        'G': 'گ',
        'D': 'D',
        'S': 'S',
        'J': 'ج',
        'W': 'د',
        'C': 'س',
        'U': 'ص',
        'R': 'ط',
        'Q': 'ق',
        'L': 'ل',
        'M': 'م',
        'N': 'ن',
        'V': 'و',
        'H': 'ه',
        'I': 'ی',
        '0': '۰',
        '1': '۱',
        '2': '۲',
        '3': '۳',
        '4': '۴',
        '5': '۵',
        '6': '۶',
        '7': '۷',
        '8': '۸',
        '9': '۹',

}
Move = {
        'A': 'الف',
        '@': 'ویلچر',
    }
for i in dataclass:
    output_file_path = f'gt_{i}.txt'
    f = open(output_file_path, 'w')
    for file in os.listdir(f'{i}/'):
        # print(files)
        # for file in files:
        error_file = ''
        if file.endswith('.xml'):
            error_file = file
            # Parse the XML file
            tree = ET.parse(f'{i}/{file}')
            root = tree.getroot()
            # Access elements in the XML tree
            filename = root.find('filename').text
            print(f'Filename: {file}')
            name = ''
            new_name=''
            # Iterate through object elements
            for obj in root.findall('object'):
                name += obj.find('name').text
            # print(name)
            for key in Move:
                if name.find(Move[key]) != -1:
                    new_name=name.replace(Move[key], key)
                    name = ''
                    name = new_name
            new_name=''
            for index, v in enumerate(name):
                if is_latin_char(v)==False and is_latin_numeral(v) == False:
                    for key in dicts:
                        if dicts[key] == v:
                            new_name=name.replace(name[index], key)
                            name = ''
                            name = new_name
            # try:
            for g in name:
                if is_latin_char(g)==False and is_latin_numeral(g) == False and g != '@':
                    name = name.replace(g, "")
                # print(is_latin_char(g), is_latin_numeral(g))
            filejpg = file.split('.')
            filesave = filejpg[0] + ".jpg"
            f.write(f'{filesave}	{name}\n')

In [ ]:
!python3 create_lmdb_dataset.py --inputPath train/ --gtFile gt_train.txt --outputPath result/train

In [ ]:
!python3 create_lmdb_dataset.py --inputPath validation/ --gtFile gt_validation.txt --outputPath result/validation

In [ ]:
!python3 train.py \
--train_data result/train --valid_data result/validation \
--select_data / --batch_ratio 1 --batch_max_length 8 --valInterval 100 --num_iter 4900 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn

In [ ]:
!python3 create_lmdb_dataset.py --inputPath test/ --gtFile gt_test.txt --outputPath result/test

In [9]:
!python3 test.py \
--eval_data result/test \
--saved_model result/TPS-ResNet-BiLSTM-Attn-Seed1111_best_accuracy.pth/best_accuracy.pth \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from result/TPS-ResNet-BiLSTM-Attn-Seed1111_best_accuracy.pth/best_accuracy.pth
dataset_root:    result/test	 dataset: /
sub-directory:	/.	 num samples: 1433
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
76.692


In [9]:
!python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder test_imag --batch_max_length 8 \
--saved_model result/TPS-ResNet-BiLSTM-Attn-Seed1111_best_accuracy.pth/best_accuracy.pth

model input parameters 32 100 20 1 512 256 38 8 TPS ResNet BiLSTM Attn
loading pretrained model from result/TPS-ResNet-BiLSTM-Attn-Seed1111_best_accuracy.pth/best_accuracy.pth
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
test_imag/00192.jpg      	67e7737                  	0.9347
test_imag/01656.jpg      	97i48912                 	0.9987
